## Siamese NN

In [2]:
import os
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split

## Preparar la data

In [3]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de entrenamiento y validación desde archivo...")
    data = np.load(data_file)
    X1_train, X1_val = data["X1_train"], data["X1_val"]
    X2_train, X2_val = data["X2_train"], data["X2_val"]
    y_train, y_val = data["y_train"], data["y_val"]
else:
    # Crear los conjuntos de datos y guardarlos
    print("Dividiendo los datos y guardándolos...")
    data_a, data_b, labels = PrepareDataSNN().process()
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
        data_a, data_b, labels, test_size=0.2, random_state=42
    )
    np.savez(data_file, X1_train=X1_train, X1_val=X1_val, 
                X2_train=X2_train, X2_val=X2_val, 
                y_train=y_train, y_val=y_val)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Cargando conjuntos de entrenamiento y validación desde archivo...
Formas de los conjuntos de entrenamiento:
X1_train: (720, 65)
X2_train: (720, 65)
y_train: (720,)

Formas de los conjuntos de validación:
X1_val: (180, 65)
X2_val: (180, 65)
y_val: (180,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [4]:
import keras
# Inicializar y entrenar la red
input_shape = (65,)  # Asumiendo que tus vectores de características tienen 65 elementos
siamese_net = SiameseNeuralNetwork(input_shape)
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")
# keras.saving.save_model(siamese_net, './models/a.keras')

Epoch 1/20


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5408 - loss: 0.8697 - val_accuracy: 0.6222 - val_loss: 0.6446
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6471 - loss: 0.6600 - val_accuracy: 0.7278 - val_loss: 0.5732
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6977 - loss: 0.5973 - val_accuracy: 0.8111 - val_loss: 0.5205
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7401 - loss: 0.5371 - val_accuracy: 0.8222 - val_loss: 0.4870
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.4462 - val_accuracy: 0.8167 - val_loss: 0.4823
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8076 - loss: 0.4317 - val_accuracy: 0.8278 - val_loss: 0.4857
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8661 - loss: 0.3572 - val_accuracy: 0.8278 - val_loss: 0.4739
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8513 - loss: 0.3603 - val_accuracy: 0.8222 - val_loss: 0.4633
Ep

## Cargar el modelo guardado

In [5]:
model = load_model('./models/model_1.keras', custom_objects={'L1Distance': L1Distance})

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │    201,600 │ input_layer_1[0]… │
│ (Sequential)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance         │ (None, 128)       │          0 │ sequential[0][0], │
│ (L1Distance)        │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        129 │ l1_distance[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 601,605 (2.29 MB)

 Trainable params: 199,937 (781.00 KB)

 Non-trainable params: 1,792 (7.00 KB)

 Optimizer params: 399,876 (1.53 MB)

## Preparar la nueva data de validacion (Proyectos de C#)

In [6]:

# Ruta para guardar los datos
data_dir = "./data_val"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de validación desde archivo...")
    data = np.load(data_file)
    X1_new, X2_new, y_new = data["X1_new"], data["X2_new"], data["y_new"]
else:
    print("Procesando los datos de validación y guardándolos...")
    data_val = PrepareDataSNN("data/features_vect_val/")
    X1_new, X2_new, y_new = data_val.process()
    
    # Guardar los datos procesados
    np.savez(data_file, X1_new=X1_new, X2_new=X2_new, y_new=y_new)


# Verificar las formas de los arrays
print("Formas de los conjuntos de validación:")
print(f"X1_new: {X1_new.shape}")
print(f"X2_new: {X2_new.shape}")
print(f"y_new: {y_new.shape}")


Cargando conjuntos de validación desde archivo...
Formas de los conjuntos de validación:
X1_new: (116, 65)
X2_new: (116, 65)
y_new: (116,)


## Evaluar el modelo con los datos nuevos

In [7]:
m = model.evaluate([X1_new, X2_new], y_new)

predictions = (model.predict([X1_new, X2_new]) > 0.5).astype(int).flatten()
        
# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_new, predictions).ravel()

# Calcular métricas adicionales
precision = precision_score(y_new, predictions)
recall = recall_score(y_new, predictions)
f1 = f1_score(y_new, predictions)

# Imprimir las métricas adicionales
print(f"Verdaderos Positivos (TP): {tp}")
print(f"Falsos Positivos (FP): {fp}")
print(f"Verdaderos Negativos (TN): {tn}")
print(f"Falsos Negativos (FN): {fn}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Exhaustividad (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7750 - loss: 0.4117  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Verdaderos Positivos (TP): 14
Falsos Positivos (FP): 14
Verdaderos Negativos (TN): 73
Falsos Negativos (FN): 15
Precisión (Precision): 0.5000
Exhaustividad (Recall): 0.4828
F1-Score: 0.4912


In [8]:
metrics = siamese_net.evaluate(X1_new, X2_new, y_new)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7750 - loss: 0.4117 
Pérdida en los nuevos datos: 0.4399
Accuracy en los nuevos datos: 0.7500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Verdaderos Positivos (TP): 14
Falsos Positivos (FP): 14
Verdaderos Negativos (TN): 73
Falsos Negativos (FN): 15
Precisión (Precision): 0.5000
Exhaustividad (Recall): 0.4828
F1-Score: 0.4912


## Hacer una predicción

In [9]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Los códigos son diferentes con una probabilidad de 0.58
La similitud entre los dos códigos es: 0.5762054324150085


## Buscando la mejor semilla para el modelo

In [10]:
import os
import tensorflow as tf
import numpy as np
import random

# Definir un rango de semillas
SEEDS = range(50)  # Probar con semillas del 0 al 49

# Variables para almacenar los mejores resultados
best_accuracy = 0
best_precision = 0
best_seed = None
best_model = None

# Datos de entrenamiento y validación
# Asegúrate de tener `X1_train`, `X2_train`, `y_train`, `X1_val`, `X2_val`, `y_val` ya definidos.

for seed in SEEDS:
    print(f"Probando con semilla: {seed}")
    
    # Configurar la semilla
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear y entrenar el modelo
    snn = SiameseNeuralNetwork(input_shape=(65,), learning_rate=0.001)
    snn.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val), epochs=10, batch_size=32)
    
    # Evaluar el modelo
    metrics = snn.evaluate(X1_new, X2_new, y_new)
    accuracy = metrics["accuracy"]
    precision = metrics["precision"]
    
    # Comparar con los mejores resultados
    if accuracy > best_accuracy or ( precision > best_precision):
        best_accuracy = accuracy
        best_precision = precision
        best_seed = seed
        best_model = snn.model  # Guardar el modelo actual como el mejor

# Imprimir los resultados
print(f"Mejor semilla: {best_seed}")
print(f"Mejor accuracy: {best_accuracy:.4f}")
print(f"Mejor precisión: {best_precision:.4f}")

# Guardar el mejor modelo
tf.keras.config.enable_unsafe_deserialization()
best_model.save("models/best_model_all_dataset_val.keras")


Probando con semilla: 0
Epoch 1/10


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4607 - loss: 0.9742 - val_accuracy: 0.5333 - val_loss: 0.6406
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5989 - loss: 0.7298 - val_accuracy: 0.5889 - val_loss: 0.5998
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6646 - loss: 0.6312 - val_accuracy: 0.5722 - val_loss: 0.5605
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7172 - loss: 0.5464 - val_accuracy: 0.6056 - val_loss: 0.5318
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7753 - loss: 0.4731 - val_accuracy: 0.6333 - val_loss: 0.5116
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8119 - loss: 0.4450 - val_accuracy: 0.7722 - val_loss: 0.4896
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8063 - loss: 0.4232 - val_accuracy: 0.8333 - val_loss: 0.4673
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8550 - loss: 0.3755 - val_accuracy: 0.8222 - val_loss: 0.4540
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5354 - loss: 0.8602 - val_accuracy: 0.5444 - val_loss: 0.6440
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5606 - loss: 0.7638 - val_accuracy: 0.6333 - val_loss: 0.5971
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6695 - loss: 0.6606 - val_accuracy: 0.7389 - val_loss: 0.5570
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7016 - loss: 0.5729 - val_accuracy: 0.7611 - val_loss: 0.5286
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7485 - loss: 0.5297 - val_accuracy: 0.7778 - val_loss: 0.5061
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8219 - loss: 0.4284 - val_accuracy: 0.7833 - val_loss: 0.4894
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8185 - loss: 0.4119 - val_accuracy: 0.8056 - val_loss: 0.4786
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8017 - loss: 0.4024 - val_accuracy: 0.8111 - val_loss: 0.4799
Ep

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5390 - loss: 0.8288 - val_accuracy: 0.6222 - val_loss: 0.6352
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5835 - loss: 0.7227 - val_accuracy: 0.6778 - val_loss: 0.5836
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6484 - loss: 0.6317 - val_accuracy: 0.6889 - val_loss: 0.5561
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7094 - loss: 0.5550 - val_accuracy: 0.7222 - val_loss: 0.5216
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7498 - loss: 0.4993 - val_accuracy: 0.7611 - val_loss: 0.4999
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7759 - loss: 0.4781 - val_accuracy: 0.7611 - val_loss: 0.4964
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7984 - loss: 0.4226 - val_accuracy: 0.8000 - val_loss: 0.4679
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8216 - loss: 0.3799 - val_accuracy: 0.8000 - val_loss: 0.4638
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4965 - loss: 0.9566 - val_accuracy: 0.7444 - val_loss: 0.6534
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6268 - loss: 0.7060 - val_accuracy: 0.7333 - val_loss: 0.5977
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6692 - loss: 0.6137 - val_accuracy: 0.7500 - val_loss: 0.5495
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7258 - loss: 0.5384 - val_accuracy: 0.7611 - val_loss: 0.5173
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7727 - loss: 0.4707 - val_accuracy: 0.7722 - val_loss: 0.4987
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7639 - loss: 0.4737 - val_accuracy: 0.7778 - val_loss: 0.4938
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8089 - loss: 0.4117 - val_accuracy: 0.7889 - val_loss: 0.4931
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8465 - loss: 0.3910 - val_accuracy: 0.7722 - val_loss: 0.4992
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5000 - loss: 0.8993 - val_accuracy: 0.5500 - val_loss: 0.6439
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6135 - loss: 0.7145 - val_accuracy: 0.6778 - val_loss: 0.5914
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6547 - loss: 0.6175 - val_accuracy: 0.6778 - val_loss: 0.5527
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7321 - loss: 0.5661 - val_accuracy: 0.6944 - val_loss: 0.5188
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7627 - loss: 0.4806 - val_accuracy: 0.7444 - val_loss: 0.4975
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7464 - loss: 0.4752 - val_accuracy: 0.7778 - val_loss: 0.4802
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8361 - loss: 0.3925 - val_accuracy: 0.7944 - val_loss: 0.4574
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8472 - loss: 0.3957 - val_accuracy: 0.8000 - val_loss: 0.4579
Ep